здесь происходит обучение двух моделей: 1 - Модель для нахождения категории вопроса, 2 - Модель для подбора ответа на вопрос (классификации ответов)

![](https://sun9-48.userapi.com/impg/apZB0dkCjgq9aT0Okr2D9KvetScDzivHxA_dQw/9QlkOXmPSys.jpg?size=1280x720&quality=96&sign=fbc29f4cebfaa8b100be2c9221f59e58&type=album)

Мы решили использовать catboost модели т.к они мало весят и быстро обучаются.

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install catboost
!pip install natasha

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 96.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=41471ed1ececfd8f28732e822f2dfb732bfcf535c4426aff7a242bad42b156f6
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26096 sha256=24b4f1248f395756397d491bd5a35d9d6debc1f60e2fbdbfddd6524697e696e8
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import catboost
import matplotlib.pyplot as plt

Класс для преобразования вопросов в вектора

In [4]:
import torch
from transformers import AutoModel, AutoTokenizer

from typing import List

class Embeddings:
    def __init__(self, model_name: str, device: torch.device) -> None:
        self.device = device
        self.model = AutoModel.from_pretrained(model_name).to(device)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

    def _average_pool(self, last_hidden_states: torch.tensor,
                 attention_mask: torch.tensor) -> torch.tensor:
        last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
        return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

    def exec(self, sentences: List[str]) -> torch.tensor:
        """Get embeddings for sentences

        Args:
            sentences (List[str]): list of sentences to embed

        Returns:
            torch.tensor: return tensor of embeddings
        """
        formatted_sentences = [f'query: {sentence}' for sentence in sentences]

        tokenized_sentences = self.tokenizer(
            formatted_sentences, max_length=512,
            padding=True, truncation=True, return_tensors='pt'
        )

        tokenized_sentences['input_ids'] = tokenized_sentences['input_ids'].to(self.device)
        tokenized_sentences['attention_mask'] = tokenized_sentences['attention_mask'].to(self.device)

        self.model.eval()
        outputs = []
        with torch.no_grad():
            for i in range(len(formatted_sentences)):
                batch = {'input_ids': tokenized_sentences['input_ids'][i].unsqueeze(0), 'attention_mask': tokenized_sentences['attention_mask'][i].unsqueeze(0)}
                out = self.model(**batch)
                embeddings = self._average_pool(out.last_hidden_state, tokenized_sentences['attention_mask'][i].unsqueeze(0))
                outputs.append(embeddings.squeeze(0))
        return torch.stack(outputs).cpu()

Загрузка данных

In [5]:
answer_class = pd.read_csv('/content/drive/MyDrive/хаки/ЦФО/answer_class.csv')

In [6]:
answer_class.head()

,answer_class,Answer
0,0,После успешного прохождения выпускных испытани...
1,1,"Чтобы получить итоговый документ, нужно сдать ..."
2,2,"Можем его выдать, если вы:\n\nоплатили обучени..."
3,3,"Можем его выдать, если вы:\n\nоплатили обучени..."
4,4,Диплом или удостоверение отправим бесплатно По...


In [7]:
data = pd.read_csv('/content/drive/MyDrive/хаки/ЦФО/aug_train_data.csv')
data['Category']=data['Category'].astype('category')


Разделение данных на тестовые и данные для обучения

In [8]:
train_data, aug_data = train_test_split(data, test_size=0.1, shuffle=True)
aug_data.reset_index(inplace=True)
train_data.reset_index(inplace=True)

Настройка катигориальных фич

In [9]:
cat_features = list(train_data.select_dtypes(include=['category']).columns)
cat_features

['Category']

In [10]:
train_data.head()

,index,Question,Category,answer_class,2024,geekbrains,ага,аккаунт,аккредитация,аттестация,...,формат,хотеть,цель,что,шаг,школа,этап,это,эффективный,ясный
0,19,Как настроить оповещения о предстоящих уроках ...,Портал,17,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,340,Какие документы необходимы для получения диплома?,Документы,2,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,103,В какие дни проходят уроки?,Расписание,22,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,381,Какой документ будет выдан по окончании обучения?,Документы,0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,341,Где на сайте GeekBrains найти список курсов дл...,Портал,14,0.0,0.508492,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Загрузка модели для создания эмбеддингов

In [11]:
import torch
device = torch.device('cuda:0')

In [12]:
embeddings = Embeddings('intfloat/multilingual-e5-large', device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [13]:
#считает эмбеддинги и записывает их в pd DataFrame
def get_embs(df):
  emb = embeddings.exec(df['Question'])
  X = pd.DataFrame(emb.numpy())
  return X

In [14]:
train_data.head()

,index,Question,Category,answer_class,2024,geekbrains,ага,аккаунт,аккредитация,аттестация,...,формат,хотеть,цель,что,шаг,школа,этап,это,эффективный,ясный
0,19,Как настроить оповещения о предстоящих уроках ...,Портал,17,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,340,Какие документы необходимы для получения диплома?,Документы,2,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,103,В какие дни проходят уроки?,Расписание,22,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,381,Какой документ будет выдан по окончании обучения?,Документы,0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,341,Где на сайте GeekBrains найти список курсов дл...,Портал,14,0.0,0.508492,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Составление train датасета для модели предсказания категорий

In [15]:
X = get_embs(train_data)
X_=train_data.drop(['Category','answer_class','Question','index'], axis = 1)
X = pd.concat([X_,X], axis = 1)
y = train_data['Category']

In [16]:
X.sample(3)

,2024,geekbrains,ага,аккаунт,аккредитация,аттестация,беспокоить,бумага,бумажный,вариант,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.566568,-0.724388,0.682871,-0.748515,-0.246289,0.749272,0.906764,-0.840130,-1.444054,0.249021
381,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.074495,-0.109593,-0.062032,-0.537617,-0.753986,0.794515,0.572092,-1.142405,0.032145,0.155784
322,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.417402,-0.274396,0.578722,-0.366040,0.339719,1.019446,0.518627,-0.668018,0.100383,-0.191644


#Модель для нахождения категории вопроса

##Обучение модели для нахождения категории вопроса

Разделение датасета на валидационную и обучающую выборку

In [17]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, shuffle=False)
from catboost import Pool

train_pool = Pool(
    X_train, y_train,
)

validation_pool = Pool(
    X_val, y_val,
)

In [18]:
model_cl_category = catboost.CatBoostClassifier(iterations=1000, learning_rate=0.05, loss_function='MultiClass', early_stopping_rounds=100, task_type='GPU')

model_cl_category.fit(train_pool, eval_set=validation_pool, verbose=100)

0:	learn: 2.1846770	test: 2.2162379	best: 2.2162379 (0)	total: 146ms	remaining: 2m 25s
100:	learn: 0.2885922	test: 0.5904778	best: 0.5904778 (100)	total: 4.94s	remaining: 44s
200:	learn: 0.1333295	test: 0.4095028	best: 0.4095028 (200)	total: 8.63s	remaining: 34.3s
300:	learn: 0.0837680	test: 0.3441863	best: 0.3441863 (300)	total: 12.9s	remaining: 29.9s
400:	learn: 0.0609606	test: 0.3136874	best: 0.3136874 (400)	total: 17.7s	remaining: 26.5s
500:	learn: 0.0454097	test: 0.2883577	best: 0.2883577 (500)	total: 21.3s	remaining: 21.2s
600:	learn: 0.0355153	test: 0.2727129	best: 0.2721030 (598)	total: 24.9s	remaining: 16.5s
700:	learn: 0.0289246	test: 0.2591629	best: 0.2591629 (700)	total: 30.3s	remaining: 12.9s
800:	learn: 0.0241627	test: 0.2528300	best: 0.2523940 (787)	total: 33.8s	remaining: 8.41s
900:	learn: 0.0209835	test: 0.2451413	best: 0.2451413 (900)	total: 37.4s	remaining: 4.11s
999:	learn: 0.0184406	test: 0.2386329	best: 0.2385690 (998)	total: 42.8s	remaining: 0us
bestTest = 0.2385

In [19]:
model_cl_category.save_model('model_cl_category.bin')

##Подсчет f1 на тестовой выборке

In [20]:
from sklearn.metrics import f1_score

Составление test датасета для модели предсказания категорий

In [21]:
X = get_embs(aug_data)
X_=aug_data.drop(['Category','answer_class','Question','index'], axis = 1)
X = pd.concat([X_,X], axis = 1)

In [22]:
y_pred = model_cl_category.predict(X)
y_true = aug_data['Category']

In [23]:
f1_score(y_pred, y_true, average='micro')

0.8695652173913043

In [24]:
torch.cuda.empty_cache()

#Модель для классификации ответов

Составление train датасета для модели предсказания ответов

In [25]:
X = get_embs(train_data)
X_=train_data.drop(['answer_class','Question','index'], axis = 1)
X = pd.concat([X_,X], axis = 1)
X['Category']=X['Category'].astype('category')


In [26]:
y = train_data['answer_class']

In [27]:
X.sample(3)

,Category,2024,geekbrains,ага,аккаунт,аккредитация,аттестация,беспокоить,бумага,бумажный,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
157,Организация уроков,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.408584,-0.516961,0.701355,0.234402,0.422189,0.466258,0.231908,-0.396933,-1.190482,-0.396648
219,Документы,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.031295,-1.362841,0.400829,-0.331380,0.111653,0.895056,0.343089,-1.057366,-1.199050,-0.096355
180,Документы,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.598482,-1.362930,0.755959,-0.361571,0.149072,0.954962,0.917444,-0.734772,-1.776577,-0.155712


##Обучение модели для предсказания ответа

In [28]:
X_train, X_val, y_train, y_val  = train_test_split(X, y, test_size=0.05, shuffle=False)
from catboost import Pool

train_pool = Pool(
    X_train, y_train,
    cat_features=cat_features,
)

validation_pool = Pool(
    X_val, y_val,
    cat_features=cat_features,
)

In [29]:
model_cl_answer = catboost.CatBoostClassifier(iterations=3000, learning_rate=0.05, loss_function='MultiClass', early_stopping_rounds=100, task_type='GPU')

model_cl_answer.fit(train_pool,  eval_set=validation_pool, verbose=100)

0:	learn: 3.3367255	test: 3.3510139	best: 3.3510139 (0)	total: 219ms	remaining: 10m 55s
100:	learn: 0.9018755	test: 1.4016690	best: 1.4016690 (100)	total: 12.2s	remaining: 5m 51s
200:	learn: 0.4119874	test: 0.9395405	best: 0.9395405 (200)	total: 25.8s	remaining: 5m 59s
300:	learn: 0.2679231	test: 0.7717166	best: 0.7717166 (300)	total: 37.5s	remaining: 5m 36s
400:	learn: 0.1915616	test: 0.6833140	best: 0.6833140 (400)	total: 49.1s	remaining: 5m 18s
500:	learn: 0.1501118	test: 0.6279302	best: 0.6279302 (500)	total: 1m	remaining: 5m 1s
600:	learn: 0.1183760	test: 0.5803888	best: 0.5803888 (600)	total: 1m 12s	remaining: 4m 47s
700:	learn: 0.0947387	test: 0.5488584	best: 0.5488584 (700)	total: 1m 23s	remaining: 4m 34s
800:	learn: 0.0764128	test: 0.5183669	best: 0.5183669 (800)	total: 1m 34s	remaining: 4m 19s
900:	learn: 0.0652048	test: 0.4956756	best: 0.4956756 (900)	total: 1m 45s	remaining: 4m 6s
1000:	learn: 0.0560349	test: 0.4791158	best: 0.4791158 (1000)	total: 1m 57s	remaining: 3m 54s


In [30]:
model_cl_answer.save_model('model_cl_answer.bin')

##Подсчет f1 на тестовой выборке

Составление test датасета для модели предсказания ответов

In [31]:
X = get_embs(aug_data)
X_=aug_data.drop(['answer_class','Question','index'], axis = 1)
X = pd.concat([X_,X], axis = 1)
X['Category']=X['Category'].astype('category')

y_pred=model_cl_answer.predict(X)
micro_f1 = f1_score(y_pred, aug_data['answer_class'], average='micro')

In [32]:
micro_f1

0.9565217391304348

#Объеденение моделей

In [33]:
def stages_pred(X, verbose=0):
  X_= get_embs(X)
  X = pd.concat([X,X_], axis = 1)
  X = X.drop(['Question','Category','answer_class','index'], axis= 1 )
  pred1=model_cl_category.predict(X)

  if verbose==1:
    print(f"Категория {pred1}")

  df=pd.DataFrame([])
  df['Category']=pred1[:,0]
  X = pd.concat([df, X], axis=1)
  X['Category']=X['Category'].astype('category')
  pred2=model_cl_answer.predict(X)
  return pred2


##Подсчет f1

In [34]:
y_pred=stages_pred(aug_data)

In [35]:
micro_f1 = f1_score(aug_data['answer_class'], y_pred, average='micro')

In [36]:
micro_f1

0.9565217391304348

#sandbox

Фунции для расчета TF-IDF

In [37]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words("russian"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [38]:
from natasha import (Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, NewsNERTagger, PER, NamesExtractor, DatesExtractor, MoneyExtractor, AddrExtractor, Doc)

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

In [39]:
def lematizate_sentance(text):
  doc = Doc(text)
  doc.segment(segmenter)
  doc.tag_morph(morph_tagger)

  new_sent=""
  for token in doc.tokens:
      token.lemmatize(morph_vocab)
      if token.lemma not in stop_words:
        new_sent += f"{token.lemma} "
  return new_sent

In [40]:
def lematizate_sentances(texts):
  new_texts=[]
  for text in texts:
    new_texts.append(lematizate_sentance(text))

  return new_texts


Фунции для полноценного фунционирования

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

# Загрузка TfidfVectorizer из файла
tfidf_vectorizer = joblib.load("/content/drive/MyDrive/хаки/ЦФО/tfidf_vectorizer.pkl")

In [42]:
#Технические заполнители, дальше будут удалены с помощью drop
def tech_fill(df):
  df['Category'] = [None]*len(df)
  df['answer_class'] = [None]*len(df)
  df['index'] = [None]*len(df)
  return df

In [43]:
def data_prep(texts):
  #Перевод списка вопросов к нужному формату
  train_data = pd.DataFrame([])
  train_data['Question']=texts
  train_data=tech_fill(train_data)

  #добавление TF-IDF
  documents=lematizate_sentances(texts)
  tfidf_matrix = tfidf_vectorizer.transform(documents)
  tfidf_table = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
  train_data=pd.concat([train_data,tfidf_table], axis=1)

  return train_data

In [44]:
text=['когда я залутаю диплом?']
#предсказание
answer=stages_pred(data_prep(text), verbose=1)
answer_class.iloc[answer[0][0]]['Answer']

Категория [['Документы']]


'Можем его выдать, если вы:\n\nоплатили обучение после 3 декабря 2019 года;\nимеете среднее или высшее профессиональное образование — подойдут диплом СПО или ВО. Если вы ещё не закончили СУЗ или ВУЗ, подойдёт справка из образовательного учреждения;\nпри зачислении предоставили пакет документов — паспорт, диплом, СНИЛС, сведения о смене ФИО (при наличии), сведения о признании иностранного диплома (при наличии);\nсдали все промежуточные аттестации до даты итоговой аттестации;\nуспешно сдали итоговую аттестацию.\nИтоговой аттестацией может быть:\n\nитоговый экзамен;\nзащита проекта;\nдругие формы итоговой аттестационной работы.\nМы подготовим диплом в течение 30 дней от даты итоговой аттестации. Чтобы его получить, обратитесь к куратору.'